# Environmental Surveillance with Google Earth Engine (GEE)

This notebook contains the code for three powerful, interactive dashboards that leverage Google Earth Engine to monitor environmental conditions related to animal health. These tools allow for proactive, data-driven surveillance by analyzing large-scale environmental datasets in near real-time.

**Dashboards Included:**
1.  **Multi-Level Environmental Surveillance Dashboard:** A tool to get a summary of key environmental risk factors (NDVI, Temperature, Livestock Density) and view anomalies for any selected administrative area.
2.  **Seasonal Drought Risk Watch Dashboard:** A proactive tool to monitor seasonal rainfall performance and identify areas at risk of drought.
3.  **Vector-Borne Disease Risk Dashboard:** An MCDA-based model that generates a dynamic environmental suitability map for disease vectors.

**Prerequisites:**
*   You must have a Google account with Google Earth Engine access.
*   Run the GEE authentication flow when prompted in the first code cell.
*   Ensure `geemap` and `earthengine-api` are installed from `requirements.txt`.

In [ ]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta

# --- Authenticate and Initialize GEE ---
try:
    ee.Initialize()
except Exception as e:
    print("GEE not initialized. Authenticating...")
    ee.Authenticate()
    ee.Initialize()

print("✅ Google Earth Engine initialized successfully.")

# --- Load Administrative Boundaries (from local files for this notebook) ---
# In a real app, this might come from a database or GEE Asset
# Creating placeholder GeoDataFrames for demonstration purposes
try:
    # This part would typically load your shapefiles or database data
    # gdf_regions = gpd.read_file('path/to/regions.shp')
    # For this script, we create a dummy representation of Ethiopia
    country_boundary_geom = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Ethiopia')).first().geometry()
    print("✅ Country boundary loaded successfully.")
except Exception as e:
    print(f"⚠️ Could not load boundary data. Using a placeholder. Error: {e}")
    country_boundary_geom = ee.Geometry.Rectangle([33, 3, 48, 15])

### Dashboard 1: Seasonal Drought Risk Watch

This dashboard calculates the total rainfall for a selected season and year and compares it to the long-term (20-year) average for that same season. The result is a 'Percent of Normal Rainfall' map, which is a key indicator for drought risk and its potential impact on forage and water availability for livestock.

In [ ]:
def calculate_seasonal_rainfall(start_year, end_year, season_months, geometry):
    collection = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
    seasonal_collection = collection.filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                                    .filter(ee.Filter.inList('month', season_months))
    return seasonal_collection.sum().clip(geometry)

# --- Widgets ---
seasons = {'Kiremt (Jun-Sep)': [6,7,8,9], 'Belg (Feb-May)': [2,3,4,5], 'Bega (Oct-Jan)': [10,11,12,1]}
season_selector = widgets.Dropdown(options=list(seasons.keys()), description='Season:')
year_selector = widgets.Dropdown(options=list(range(2000, 2025)), value=2023, description='Year:')
run_button_drought = widgets.Button(description="Generate Drought Watch", button_style='success')
map_output_drought = widgets.Output()

def update_drought_dashboard(b):
    with map_output_drought:
        clear_output(wait=True); print("Running analysis...")
        selected_year = year_selector.value
        season_months = seasons[season_selector.value]
        current = calculate_seasonal_rainfall(selected_year, selected_year, season_months, country_boundary_geom)
        long_term_avg = calculate_seasonal_rainfall(2000, 2020, season_months, country_boundary_geom).divide(21)
        percent_of_normal = current.divide(long_term_avg).multiply(100)
        
        center = country_boundary_geom.centroid(1).getInfo()['coordinates']
        Map = geemap.Map(center=[center[1], center[0]], zoom=6)
        palette = ['#FF0000', '#FFA500', '#FFFFFF', '#ADFF2F', '#008000']
        vis_params = {'min': 50, 'max': 150, 'palette': palette}
        Map.addLayer(percent_of_normal, vis_params, f'Rainfall Anomaly ({selected_year})')
        Map.add_colorbar_with_label(vis_params=vis_params, label="Percent of Normal Rainfall (%)")
        display(Map)

run_button_drought.on_click(update_drought_dashboard)
drought_dashboard = widgets.VBox([
    widgets.HTML("<h2>Seasonal Rainfall Anomaly Dashboard</h2>"),
    widgets.HBox([season_selector, year_selector, run_button_drought]),
    map_output_drought
])
print("✅ Drought Risk Dashboard is ready.")
display(drought_dashboard)

### Dashboard 2: Vector-Borne Disease Risk Watch

This dashboard provides a dynamic environmental suitability assessment for disease vectors (e.g., mosquitoes, midges). It uses a Multi-Criteria Decision Analysis (MCDA) approach, combining temperature, rainfall, and vegetation data into a single 'Vector Suitability Index'. This provides an early warning for where conditions are most favorable for vector proliferation.

In [ ]:
# --- Widgets ---
date_selector_vector = widgets.DatePicker(description='Select Date:', value=date.today() - timedelta(weeks=4))
run_button_vector = widgets.Button(description="Generate Vector Risk Map", button_style='primary')
map_output_vector = widgets.Output()
summary_output_vector = widgets.HTML()

def update_vector_dashboard(b):
    with map_output_vector:
        clear_output(wait=True); print("Running analysis...")
        target_date = ee.Date(date_selector_vector.value.strftime('%Y-%m-%d'))
        date_range = ee.DateRange(target_date.advance(-30, 'day'), target_date)
        
        # MCDA logic
        w_temp, w_precip, w_ndvi = 0.4, 0.35, 0.25
        temp_c = ee.ImageCollection('MODIS/061/MOD11A2').filterDate(date_range).select('LST_Day_1km').mean().multiply(0.02).subtract(273.15)
        temp_risk = temp_c.gte(20).And(temp_c.lte(30))
        precip = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(date_range).select('precipitation').sum()
        precip_risk = precip.unitScale(0, 200) # Normalize based on expected range
        ndvi = ee.ImageCollection('MODIS/061/MOD13A1').filterDate(date_range).select('NDVI').mean()
        ndvi_risk = ndvi.unitScale(1000, 8000) # Normalize based on expected range
        composite_risk = (temp_risk.multiply(w_temp)).add(precip_risk.multiply(w_precip)).add(ndvi_risk.multiply(w_ndvi))
        
        center = country_boundary_geom.centroid(1).getInfo()['coordinates']
        Map = geemap.Map(center=[center[1], center[0]], zoom=6)
        vis_params = {'min': 0, 'max': 1, 'palette': ['#440154', '#3b528b', '#21918c', '#5ec962', '#fde725']}
        Map.addLayer(composite_risk.clip(country_boundary_geom), vis_params, 'Vector Suitability Index')
        Map.add_colorbar_with_label(vis_params=vis_params, label="Vector Suitability Index")
        Map.addLayer(temp_risk.selfMask(), {'palette': ['red']}, 'Optimal Temperature', False)
        display(Map)
        summary_output_vector.value = f"<h4>Vector Suitability for {date_selector_vector.value.strftime('%B %d, %Y')}</h4>"

run_button_vector.on_click(update_vector_dashboard)
vector_dashboard = widgets.VBox([
    widgets.HTML("<h2>Vector Suitability and Risk Watch Dashboard</h2>"),
    widgets.HBox([date_selector_vector, run_button_vector]),
    summary_output_vector,
    map_output_vector
])
print("✅ Vector-Borne Disease Dashboard is ready.")
display(vector_dashboard)